In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability.python.bijectors import affine_scalar
import math as m
import random as r
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import h5py
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import math
from keras import backend as K

%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import math
import sys

from keras.layers import Lambda, Input, Dense, LeakyReLU, BatchNormalization, Concatenate, Reshape, Conv1D
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import regularizers
from scipy.stats import multivariate_normal
from keras.optimizers import Adam
import tensorflow as tf

from sklearn.metrics import log_loss, roc_curve

from glob import glob
from tqdm import tqdm_notebook as tqdm

%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, roc_auc_score
from keras.datasets import mnist
import importlib
import h5py

In [ ]:
SIG_NAMES = {
    '1': {
        1: "stop2b1000_neutralino300",
        2: "glgl1400_neutralino1100",
        28: "monojet_Zp2000.0_DM_50.0",
        29: "glgl1600_neutralino800",
        30: "monotop_200_A",
        31: "stlp_st1000",
        32: "sqsq_sq1800_neut800",
        33: "sqsq1_sq1400_neut800",
        999: "Secret set"
    },
    '2a': {
        25: "chaneut_cha250_neut150",
        26: "chaneut_cha400_neut200",
        27: "pp24mt_50",
        28: "pp23mt_50",
        29: "gluino_1000.0_neutralino_1.0",
        30: "chaneut_cha200_neut50",
        31: "chaneut_cha300_neut100",
        999: "Secret set"
    },
    '2b': {
        1: "pp24mt_50",
        2: "chaneut_cha200_neut50",
        3: "stlp_st1000",
        4: "chacha_cha600_neut200",
        5: "pp23mt_50",
        6: "chaneut_cha250_neut150",
        7: "chacha_cha400_neut60",
        34: "gluino_1000.0_neutralino_1.0",
        35: "chacha_cha300_neut140",
        999: "Secret set"
    },
    '3': {
        1: "glgl1600_neutralino800",
        2: "monojet_Zp2000.0_DM_50.0",
        3: "gluino_1000.0_neutralino_1.0",
        4: "stop2b1000_neutralino300",
        5: "sqsq1_sq1400_neut800",
        6: "monotop_200_A",
        7: "monoV_Zp2000.0_DM_1.0",
        8: "stlp_st1000",
        34: "sqsq_sq1800_neut800",
        35: "glgl1400_neutralino1100",
        999: "Secret set"
    }
}

In [ ]:
SIG_TYPES = {
    '1': [1, 2, 28, 29, 30, 31, 32, 33, 999],
    '2a': [25, 26, 27, 28, 29, 30, 31, 999],
    '2b': [1,2,3,4,5,6,7, 34,35, 999],
    '3': [1,2,3,4,5,6,7,8,34,35, 999]
}

"""
class types:
0 none
1 j
2 b
3 g
4 e+
5 e-
6 m+
7 m-
"""

CLASS = {
    'none': 0.,
    'j': 1.,
    'b': 2.,
    'g': 3.,
    'e+': 4.,
    'e-': 5.,
    'm+': 6.,
    'm-': 7.
}
MAX_JET = 7
MAX_LEP = 4

CHANNELS = ['3']
n_epochs = 100

def get_data(channel):
    train_h5 = h5py.File('../hackathon-anomaly/training_chan'+str(channel)+'.h5','r')
    train = [train_h5['clas'][:,:,0], train_h5['reg'][:]]
    
    test_h5 = h5py.File('../hackathon-anomaly/testing_chan'+str(channel)+'.h5', 'r')
    secret_h5 = h5py.File('../hackathon-anomaly/inference_chan'+str(channel)+'.h5', 'r')
    
    test_bg_clas = []
    test_bg_reg = []
    
    test_sig_clas = []
    test_sig_reg = []
    
    sig_types = []
    
    
    types = test_h5['type']
    clas = test_h5['clas']
    reg = test_h5['reg']
    
    for i in tqdm(range(len(types))):
        typ = types[i]
        if typ in SIG_TYPES[channel]:
            # signal
            test_sig_clas.append(clas[i,:,0])
            test_sig_reg.append(reg[i])
            sig_types.append(typ)
        else:
            # bg
            test_bg_clas.append(clas[i,:,0])
            test_bg_reg.append(reg[i])
            
    # Append secret set
    secret_clas = secret_h5['clas']
    secret_reg = secret_h5['reg']
    for i in tqdm(range(len(secret_reg))):
        sig_types.append(999)
        test_sig_clas.append(secret_clas[i,:,0])
        test_sig_reg.append(secret_reg[i])
    
    test_bg = [np.array(test_bg_clas), np.array(test_bg_reg)]
    test_sig = [np.array(test_sig_clas), np.array(test_sig_reg)]
        
    return train, test_bg, test_sig, sig_types


train, test_bg, test_sig, signal_types = get_data('2a')
print(train[0])
print('done')

In [ ]:

def get_R(coords, center=None):
    return np.linalg.norm(coords,axis=1)

def append_to_result_radius(result_name, scores):
    f = open("results-radius.csv", "a+")
    for typ in scores:
        f.write(result_name + "," + str(typ) + "," + str(scores[typ]) + "\n")

def save_model_results(model_name, model, test_bg, test_sig, sig_types):
    # Train results
    model.set_latent_mean(train)
    model.save_training_scores(model_name, train)
    
    # Radius
    model.evaluate_radius(test_bg, test_sig)
    append_to_result_radius(model_name, model.score(sig_types))
    
    # Save results
    model.save_scores(model_name)
    
def save_model_scores(model_name, model, test_bg, test_sig, sig_types):
    # Train results
    model.set_latent_mean(train)
    model.save_training_scores(model_name, train)
    model.evaluate_radius(test_bg, test_sig)
    model.score_individual(sig_types)
    append_to_result_radius(model_name, model.score(sig_types))
    


def determine_auc_score(test_bg_scores, test_sig_scores):
    test_sig_scores = np.array(test_sig_scores)
    y_true = np.concatenate((np.zeros(test_bg_scores.shape[0]), np.ones(test_sig_scores.shape[0])))
    y_test = np.concatenate((test_bg_scores, test_sig_scores))
    return roc_auc_score(y_true, y_test)

class VariationalAutoencoderModel():
    def __init__(self, filename, D, dim_z, loss_fn, verbose=False):
        self.D = D
        self.dim_z = dim_z
        self.verbose = verbose
        self.model_filename = 'models/' + filename + '.h5'
        self.filename = filename
        self.loss_fn = loss_fn
        
        model = self.build_lhcdata_model()

        if self.verbose:
            model.summary()
            
        self.model = model
        return
    
    def z_log_var_activation(self, x):
        return K.sigmoid(x) * 10

    def build_lhcdata_model(self):
        D = self.D
        in_classification = Input(shape=(D,), name='in_classification')
        in_regression = Input(shape=(D*4+2,), name='in_regression')
        inputs = Concatenate()([in_classification, in_regression])

        x = Dense(512, activation='elu')(inputs)
        x = Dense(256, activation='elu')(x)
        x = Dense(128, activation='elu')(x)
        z_mean = Dense(self.dim_z, name='z_mean', activation='linear')(x)

        # instantiate encoder model
        self.encoder = Model([in_classification, in_regression], [z_mean], name='encoder')

        self.encoder.compile(optimizer='adam', loss='mean_squared_error')
        return self.encoder
    
    def train_model(self, train, batch_size=10000):
        earlystopper = EarlyStopping(monitor='loss', patience=50, verbose=0, min_delta=1e-7)
        checkpointer = ModelCheckpoint(self.model_filename, monitor='loss', verbose=1, save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.000000001)
        lr = 1e-3
        epoch_num = 0
        best_loss = 1e10
        cur_patience = 0
        max_patience_lr = 5
        max_patience_quit = 10
        
        # Or have 2 output arms with the same output that have different inputs and train like that?
        # Or latent space that is much hgiher than inut space, as this problem is much
        # more complex?
            
        res_train = self.model.fit(
            train, 
            self.loss_fn(train[0].shape[0], self.dim_z),
            batch_size=batch_size, 
            epochs=10000,
            shuffle=True,
            verbose=1 if self.verbose else 0, 
            callbacks=[earlystopper, reduce_lr, checkpointer]
        )
        np.savetxt('losses/' + self.filename, res_train.history['loss'])
    
    def load_weights(self, weight_filename):
        self.model.load_weights(weight_filename)
        
    def set_latent_mean(self, _):
        return
    
    def evaluate_radius(self, test_bg, test_sig, batch_size=10000):
        latent_space_bg = self.encoder.predict(test_bg, batch_size=batch_size, verbose=self.verbose)
        latent_space_sig = self.encoder.predict(test_sig, batch_size=batch_size, verbose=self.verbose)
        
        test_bg_scores = get_R(latent_space_bg - self.loss_fn(latent_space_bg.shape[0], self.dim_z))
        test_sig_scores = get_R(latent_space_sig - self.loss_fn(latent_space_sig.shape[0], self.dim_z))
        
        self.radius_bg = test_bg_scores
        self.radius_sig = test_sig_scores
        
        self.test_bg_scores_r = test_bg_scores / self.max_radius
        self.test_sig_scores_r = test_sig_scores / self.max_radius
        return test_bg_scores, test_sig_scores
    
    def save_scores(self, model_name):
        np.savetxt('scores/radius-bg-' + model_name, self.test_bg_scores_r)
        np.savetxt('scores/radius-sig-' + model_name, self.test_sig_scores_r)
        
    def save_training_scores(self, model_name, train, batch_size=10000):
        latent_space_train = self.encoder.predict(train, batch_size=batch_size, verbose=self.verbose)
        train_scores_radius = get_R(latent_space_train - self.loss_fn(latent_space_train.shape[0], self.dim_z))
        self.max_radius = np.max(train_scores_radius)
        np.savetxt('scores/radius-train-' + model_name, train_scores_radius / self.max_radius)


    
    def score(self, sig_types):
        sig_scores = {}
        auc_scores = {}
        unique_types = np.unique(sig_types)
        for typ in unique_types:
            sig_scores[typ] = []
        
        for i in range(len(self.test_sig_scores_r)):
            sig_scores[sig_types[i]].append(self.test_sig_scores_r[i])
            
#         plt.subplot(1,3,1)
#         plt.title("background")
#         plt.hist(self.test_bg_scores_r, bins=20)
#         plt.subplot(1,3,2)
#         plt.title("signal gluino_1000.0")
#         plt.hist(sig_scores[30], bins=20)
#         plt.subplot(1,3,3)
#         plt.title("Secret")
#         plt.hist(sig_scores[999], bins=20)
#         asasdasd
            
        for typ in unique_types:
            auc_scores[typ] = determine_auc_score(self.test_bg_scores_r, sig_scores[typ])
        return auc_scores
        
    def score_individual(self, sig_types):
        sig_scores = {}
        auc_scores = {}
        unique_types = np.unique(sig_types)
        for typ in unique_types:
            sig_scores[typ] = []
        
        for i in range(len(self.test_sig_scores_r)):
            sig_scores[sig_types[i]].append(self.test_sig_scores_r[i])
        for typ in unique_types:
            auc_scores[typ] = determine_auc_score(self.test_bg_scores_r, sig_scores[typ])
            np.savetxt('best_scores/radius-bg-' + model_name + '-' + str(typ), self.test_bg_scores_r)
            np.savetxt('best_scores/radius-sig-' + model_name + '-' + str(typ), sig_scores[typ])
        return auc_scores

        

In [ ]:
def MSEc(c):
    def MSE(dataset_len, dim_z):
        output_data = np.zeros((dataset_len, dim_z), np.int32)
        output_data.fill(c)
        return output_data
    return MSE

In [ ]:
dim_z = [5,8,13,21,34,55,89,144,233]

In [ ]:

def run_experiments(name, loss_fn):
    for c in CHANNELS:
        print(c, name)
        global train
        global test_bg
        global test_sig
        global sig_types
        global model_name
        train, test_bg, test_sig, sig_types = get_data(c)
        for _z in dim_z:
            with tf.device("/gpu:0"):
                sess = tf.Session()
                K.set_session(sess)

                model_name = name + '_' + c + '_' + str(_z)
                model = VariationalAutoencoderModel(model_name, train[0].shape[1], _z, loss_fn)
                model.train_model(train)
#                 model.load_weights('models/'+model_name+'.h5')
                save_model_scores(model_name, model, test_bg, test_sig, sig_types)
CHANNELS = ['1','2a','2b','3']
for i in range(1):#range(5):
    run_experiments('MSE0-run'+str(i), MSEc(0))
    run_experiments('MSE1-run'+str(i), MSEc(1))
    run_experiments('MSE2-run'+str(i), MSEc(2))
    run_experiments('MSE3-run'+str(i), MSEc(3))
    run_experiments('MSE4-run'+str(i), MSEc(4))
    run_experiments('MSE10-run'+str(i), MSEc(10))
    run_experiments('MSE25-run'+str(i), MSEc(25))
